In [ ]:
import yfinance as yf
from datetime import datetime
from utils import Singleton, TickerCache
import ticker_utils as tu

# yf.enable_debug_mode()

def analyse(ticker_data):
    interval_text = tu.get_interval_text(ticker_data)

    latest_market_price = tu.get_latest_close(ticker_data)
    highest = tu.get_high_market_price(ticker_data)
    lowest = tu.get_low_market_price(ticker_data)

    data_of_current_year = ticker_data[ticker_data.index.year == datetime.now().year]
    start_market_price = data_of_current_year.iloc[0]["Open"]
    performance = ((latest_market_price.item() - start_market_price.item()) / start_market_price.item()) * 100

    print(f"Aktueller Schlusskurs ({ticker_data.index[-1].date()}): ${latest_market_price.item()}")
    print(f"Historischer Höchstkurs ({interval_text}): ${highest[0]} am {highest[1].strftime('%d.%m.%Y')}")
    print(f"Historischer Tiefstkurs ({interval_text}): ${lowest[0]} am {lowest[1].strftime('%d.%m.%Y')}")
    print(f"Performance seit Jahresbeginn (YTD): {performance:.2f}% (Startkurs: ${start_market_price.item()} - Schlusskurs: ${latest_market_price.item()})")

class TickerWrapper(Singleton):
    __ticker_cache = TickerCache()

    def __init__(self, ttl_minutes=3):
        self.__ticker_cache = TickerCache(ttl_minutes)
        pass

    def get_ticker_data(self, tickers: str | list[str], period='1y'):
        """
        Gets the stock data for the provided tickers for the provided period

        Parameters
        ----------
        tickers : str | list of str
            The ticker symbols to get data for
        period: str, optional
            The period to get the data for. Default is 1-year
        Returns
        -------
        dict[str, Any | None]
            The stock data for the provided tickers
        """
        if not tickers:
            print(f"Bitte mindestens einen Ticker angeben.")
            return None

        if isinstance(tickers, str):
            tickers = [tickers]

        data = {ticker: self.__ticker_cache.get(ticker, period) for ticker in tickers}
        missing_tickers = [key for key, value in data.items() if value is None]

        period = tu.get_next_suitable_period(period) or "1y"

        if missing_tickers:
            try:
                fetched_data = yf.download(missing_tickers, period=period, group_by='ticker', rounding=True)
            except Exception as e:
                print(f"Fehler beim Abruf der Daten: {e}")
                return None

            if len(missing_tickers) == 1 and missing_tickers[0] not in fetched_data.columns:
                updated_data = {missing_tickers[0]: fetched_data}
            else:
                updated_data = {t: fetched_data[t] for t in missing_tickers}
            data.update(updated_data)
            self.__ticker_cache.set_tickers(updated_data, period)

        return data

def main():
    my_ticker = TickerWrapper()

    while True:
        ticker_input = input(
            "------------------------------------------------------------\n"
            "Welche Ticker sollen angezeigt werden? (z.B. AAPL, NVDA): "
        )
        if not ticker_input:
            break

        ticker_period = input(
            "Für welchen Zeitraum in Monate? "
            "(keine oder eine ungültige Angabe ruft die Daten der letzten 12 Monate ab)"
        )
        if not ticker_period.isdigit():
            ticker_period = "12"
        ticker_period += "mo"

        try:
            ticker_data = my_ticker.get_ticker_data(
                [ticker.strip().upper() for ticker in ticker_input.split(",")],
                period=ticker_period
            )
            for ticker in ticker_data:
                print(f"\nAktuelle Daten für {ticker}")
                analyse(ticker_data[ticker])
        except Exception:
            print("Fehler beim Abfragen der Daten")


if __name__ == "__main__":
    main()

C:\Users\a_hic\AppData\Local\Temp\ipykernel_15936\273375620.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fetched_data = yf.download(missing_tickers, period=period, group_by='ticker', rounding=True)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['APPL']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')


found period: 1y

Aktuelle Daten für APPL
Fehler beim Abfragen der Daten


C:\Users\a_hic\AppData\Local\Temp\ipykernel_15936\273375620.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  fetched_data = yf.download(missing_tickers, period=period, group_by='ticker', rounding=True)
[*********************100%***********************]  1 of 1 completed

found period: 1y

Aktuelle Daten für AAPL
Aktueller Schlusskurs (2026-01-28): $256.44
Historischer Höchstkurs (vom 29.01.2025 bis 28.01.2026): $288.62 am 03.12.2025
Historischer Tiefstkurs (vom 29.01.2025 bis 28.01.2026): $168.63 am 08.04.2025
Performance seit Jahresbeginn (YTD): -5.81% (Startkurs: $272.26 - Schlusskurs: $256.44)
